<a href="https://colab.research.google.com/github/musab110/Image_Processing_Project/blob/main/Copy_YOLO_Thai_OCR_Trainingipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div dir="rtl" style="text-align: right; font-family: 'Arial', sans-serif;">

# مشروع OCR للغة التايلاندية: من البيانات إلى النموذج الخبير
---
**إعداد الطلاب:** مصعب الجعشني - سليمان الجبرتي  
**إشراف الدكتور:** أكرم الصباري



## المرحلة1: الإعدادات الأولية وتفعيل الـ GPU
<p style="font-size: 16px;"><b>(يجب تشغيلها عند العودة)</b></p>

في هذه المرحلة، نتأكد من أننا نستخدم البيئة الصحيحة (GPU) للقيام بالعمليات الحسابية اللازمة.

</div>

In [ ]:
# التحقق من وجود GPU وتخصيصه لهذه الجلسة
!nvidia-smi

Thu Oct  2 08:17:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

#### **المرحلة2: تجهيز بيئة العمل**

<div dir="rtl" style="text-align: right; font-family: 'Arial', sans-serif;">

## المرحلة2: تجهيز بيئة العمل (Setup)
<p style="font-size: 16px;"><b>(يجب تشغيلها عند العودة)</b></p>

نقوم بالاتصال بـ Google Drive وتثبيت كل المكتبات التي سنحتاجها (`ultralytics` لنماذج YOLO، و`easyocr` لقراءة النصوص، و`gradio` للواجهة التفاعلية).

</div>

In [ ]:
# 1. الاتصال بـ Google Drive
from google.colab import drive
print("--> جاري الاتصال بـ Google Drive...")
drive.mount('/content/drive')

# 2. تثبيت المكتبات
print("\n--> جاري تثبيت المكتبات اللازمة...")
!pip install ultralytics easyocr gradio -q
print("✅ اكتمل تجهيز بيئة العمل بنجاح!")

--> جاري الاتصال بـ Google Drive...
Mounted at /content/drive

--> جاري تثبيت المكتبات اللازمة...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 112.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 26.0 MB/s eta 0:00:00
✅ اكتمل تجهيز بيئة العمل بنجاح!


In [ ]:
# 1. الاتصال بـ Google Drive
from google.colab import drive
print("--> جاري الاتصال بـ Google Drive...")
drive.mount('/content/drive')

# 2. تثبيت المكتبات
print("\n--> جاري تثبيت المكتبات اللازمة...")
!pip install ultralytics easyocr gradio -q
print("✅ اكتمل تجهيز بيئة العمل بنجاح!")

--> جاري الاتصال بـ Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

--> جاري تثبيت المكتبات اللازمة...
✅ اكتمل تجهيز بيئة العمل بنجاح!


#### **المرحلة3: اختبار النموذج الأصلي (v1)**


<div dir="rtl" style="text-align: right; font-family: 'Arial', sans-serif;">

## المرحلة 3: اختبار النموذج الأصلي (v1)
<p style="font-size: 16px;"><b>(يجب تشغيلها عند العودة)</b></p>

سنقوم بتحميل النموذج الأول الذي تدرب على 5000 صورة نظيفة (`best.pt`). ثم سنختبره على صورة صعبة من العالم الحقيقي لنرى أداءه الأساسي ونحدد نقاط ضعفه.

</div>

In [ ]:
# 1. تحميل النماذج
from ultralytics import YOLO
import easyocr
print("--> تحميل النموذج الأصلي (v1) وقارئ النصوص...")
try:
    # !! تأكد من أن هذا هو اسم النموذج الأول الذي حفظته !!
    model_v1 = YOLO("/content/drive/MyDrive/yolo_ocr_project/best.pt")
    reader = easyocr.Reader(['th', 'en'])
    print("✅ تم تحميل النموذج الأصلي وقارئ النصوص بنجاح!")
except Exception as e:
    print(f"❌ خطأ: {e}")

# 2. كود الاختبار التفاعلي للنموذج الأصلي
import gradio as gr
from PIL import Image
import cv2
import numpy as np

def test_model(input_image, model_to_test):
    if input_image is None: return None
    results = model_to_test(input_image)
    img_cv = cv2.cvtColor(np.array(input_image), cv2.COLOR_RGB2BGR)
    for box in results[0].boxes:
        xyxy = box.xyxy[0].cpu().numpy().astype(int)
        cv2.rectangle(img_cv, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0, 0, 255), 3) # مربع أزرق
    return Image.fromarray(cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB))

with gr.Blocks() as demo_v1:
    gr.Markdown("### اختبار النموذج الأصلي (v1)")
    with gr.Row():
        image_input_v1 = gr.Image(type="pil", label="ارفع صورة صعبة هنا")
        image_output_v1 = gr.Image(label="نتيجة الكشف من v1")
    image_input_v1.change(fn=test_model, inputs=[image_input_v1, gr.State(model_v1)], outputs=image_output_v1)

print("\n--- الواجهة جاهزة لاختبار النموذج الأصلي (v1) ---")
demo_v1.launch()

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
--> تحميل النموذج الأصلي (v1) وقارئ النصوص...


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete✅ تم تحميل النموذج الأصلي وقارئ النصوص بنجاح!


KeyboardInterrupt: 

#### **المرحلة 4: التدريب المُحسّن (v2) - بناء النموذج الخبير**

<div dir="rtl" style="text-align: right; font-family: 'Arial', sans-serif;">

## المرحلة4: التدريب المتقدم (v2) - بناء النموذج الخبير

**التحليل:** لاحظنا أن النموذج الأصلي (v1) يرتكب أخطاء في الصور المعقدة.  
**الحل:** سنقوم الآن بعملية **ضبط دقيق نهائية**. سنبدأ من النموذج v1، ولكن سندربه على **"حزمة البيانات الخارقة"** التي تحتوي على كل شيء (الصور الأصلية + السلبية + الاحترافية) لتعليمه كيفية التعامل مع العالم الحقيقي.

<p style="font-size: 16px;"><b>(هذه الخلية تقوم بالتدريب، لا تحتاج لتشغيلها في كل مرة، فقط عند الحاجة لإعادة التدريب)</b></p>

</div>

In [ ]:
# 1. فك ضغط حزمة البيانات الخارقة من Google Drive
print("--> فك ضغط البيانات النهائية والمدمجة...")
# !! تأكد من أن هذا هو ملف الـ zip الأخير الذي رفعته !!
zip_file_path = "/content/drive/MyDrive/yolo_ocr_project/02_dataset_output.zip"
!unzip -o -q "{zip_file_path}" -d /content/
print("✅ اكتمل فك الضغط!")

# 2. إنشاء ملف data.yaml
import yaml
data_yaml_content = {'train': '/content/02_dataset_output/images/train', 'val': '/content/02_dataset_output/images/val', 'nc': 1, 'names': ['text']}
file_path = '/content/02_dataset_output/data.yaml'
with open(file_path, 'w') as f:
    yaml.dump(data_yaml_content, f, sort_keys=False)
print("✅ تم إنشاء ملف data.yaml.")

# 3. بدء التدريب الأخير لبناء النموذج الخبير
print("\n--> بدء عملية الضبط الدقيق النهائية لـ 40 دورة...")
!yolo task=detect mode=train \
model=/content/drive/MyDrive/yolo_ocr_project/best.pt \
data=/content/02_dataset_output/data.yaml \
epochs=40 \
imgsz=640 \
batch=8

--> فك ضغط البيانات النهائية والمدمجة...
✅ اكتمل فك الضغط!
✅ تم إنشاء ملف data.yaml.

--> بدء عملية الضبط الدقيق النهائية لـ 40 دورة...
Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/02_dataset_output/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/MyDrive/yolo_ocr_project/best.pt, momentum=0

#### **المرحلة 5: العرض النهائي للنموذج الخبير (v2)**

<div dir="rtl" style="text-align: right; font-family: 'Arial', sans-serif;">

## المرحلة 5: العرض النهائي للنموذج الخبير (v2)
<p style="font-size: 16px;"><b>(يجب تشغيلها عند العودة، بعد تشغيل المرحلتين 0 و 1)</b></p>

بعد انتهاء التدريب الأخير، أصبح لدينا الآن "نموذج خبير". سنقوم بحفظه، ثم نختبره باستخدام الواجهة التفاعلية الاحترافية على أصعب الصور لنرى التحسن المذهل في الأداء.

</div>

In [ ]:
# 1. حفظ النموذج الخبير
print("--> حفظ النموذج الخبير في Google Drive...")
# !! انتبه: قد تحتاج إلى تغيير الرقم (مثلاً train... ) حسب رقم المجلد الذي أنشأه YOLO !!
last_run_path = "/content/runs/detect/train/weights/best.pt" # افترضنا أن اسمه train، عدّله إذا كان مختلفًا
destination_path = "/content/drive/MyDrive/yolo_ocr_project/FINAL_EXPERT_MODEL.pt"
!cp -f "{last_run_path}" "{destination_path}" # استخدام -f للكتابة فوق الملف القديم
print(f"✅ تم حفظ النموذج الخبير باسم 'FINAL_EXPERT_MODEL.pt'!")

# 2. تحميل النموذج الخبير
print("\n--> تحميل النموذج الخبير...")
model_expert = YOLO(destination_path)
print("✅ النموذج الخبير جاهز!")

# 3. إطلاق الواجهة التفاعلية النهائية
import gradio as gr
from PIL import Image, ImageDraw
import cv2
import numpy as np

def ocr_expert_system(input_image):
    if input_image is None:
        empty_image = Image.new('RGB', (500, 300), color=(240, 240, 240))
        draw = ImageDraw.Draw(empty_image)
        draw.text((150, 140), "الرجاء رفع صورة أولاً", fill="grey")
        return empty_image, "لم يتم رفع أي صورة."

    img_cv = cv2.cvtColor(np.array(input_image), cv2.COLOR_RGB2BGR)
    results = model_expert(input_image)
    all_extracted_texts = []

    for box in results[0].boxes:
        xyxy = box.xyxy[0].cpu().numpy().astype(int)
        x1, y1, x2, y2 = xyxy
        cropped_image = img_cv[y1:y2, x1:x2]

        scale_factor = 3
        width = int(cropped_image.shape[1] * scale_factor)
        height = int(cropped_image.shape[0] * scale_factor)
        upscaled_cropped = cv2.resize(cropped_image, (width, height), interpolation=cv2.INTER_CUBIC)

        ocr_result = reader.readtext(upscaled_cropped, detail=0, paragraph=True)
        if ocr_result:
            detected_text = ' '.join(ocr_result)
            all_extracted_texts.append(detected_text)
            cv2.rectangle(img_cv, (x1, y1), (x2, y2), (0, 255, 0), 3)

    final_image_pil = Image.fromarray(cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB))
    final_text_output = "\n".join([f"- {text}" for text in all_extracted_texts]) if all_extracted_texts else "لم يتم العثور على أي نص في الصورة."
    return final_image_pil, final_text_output

with gr.Blocks(theme=gr.themes.Glass()) as demo_expert:
    gr.Markdown("<h1>🇹🇭 نظام OCR الخبير للغة التايلاندية 🇹🇭</h1>")
    with gr.Row():
        with gr.Column(scale=1):
            image_input = gr.Image(type="pil", label="ارفع صورتك الصعبة هنا")
            submit_button = gr.Button("🚀 ابدأ التحليل بالنموذج الخبير", variant="primary")
        with gr.Column(scale=1):
            image_output = gr.Image(label="النتائج النهائية")
            text_output = gr.Textbox(label="النص المستخرج", lines=8)
    submit_button.click(fn=ocr_expert_system, inputs=image_input, outputs=[image_output, text_output])

print("\n--- الواجهة جاهزة لاختبار النموذج الخبير النهائي ---")
demo_expert.launch(debug=True, share=True)

--> حفظ النموذج الخبير في Google Drive...
cp: cannot stat '/content/runs/detect/train/weights/best.pt': No such file or directory
✅ تم حفظ النموذج الخبير باسم 'FINAL_EXPERT_MODEL.pt'!

--> تحميل النموذج الخبير...
✅ النموذج الخبير جاهز!

--- الواجهة جاهزة لاختبار النموذج الخبير النهائي ---
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1bb9654ada64947ffe.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



0: 640x640 5 texts, 7.3ms
Speed: 20.9ms preprocess, 7.3ms inference, 232.4ms postprocess per image at shape (1, 3, 640, 640)
